#### 아래 내용은 Intel® AI for Youth 프로그램을 참고하여
BrainAI와 BrainAI Coach Network에서 개발한 내용입니다.<br> 
상업적 사용은 불가하며 교육기관 및 학교에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: BrainAI Car 자율 주행
직진, 좌회전, 정지 등의 신호를 인식하는 자율 주행 테스트

### Task 1. 필요 라이브러리 불러오기

In [4]:
import cv2
import numpy as np
import time
 
from utils import serialwrite as sw
from utils import webcam, brain, sign
from utils import linetracing as lt

### Task 2. Main 함수 작성

In [5]:
def Main():
    
    MODE_PAUSE = -1 
    MODE_LINE_TRACING = 0
    MODE_CROSSWALK = 1
    MODE_WAIT = 2
    MODE_INFERENCE = 3
    MODE_MOVE = 4
    MODE_FORWARD = 5
    mode_status = MODE_PAUSE
    
    video_capture = cv2.VideoCapture(video)
    video_capture.set(cv2.CAP_PROP_BUFFERSIZE, 0)

    ser = sw.startSerial(serial_on)
    cnn_model = brain.CNN_Model()
    model = cnn_model.load_CNN_Model()
    labels = cnn_model.GetLabels()
    
    text = "Press 'p' to start"
    color = (0, 0, 255)
    turn = 90
    box = None
    tic = time.time()
    
    print("Press 'q' to quit") 
    print("Press 'p' to start/pause")
    print()
    
    while(True):     
        grabbed, frame = video_capture.read()
        cropFrame, y1, y2, width, height = webcam.Crop(frame)
        
        if mode_status == MODE_LINE_TRACING:
            text = ""
            box = lt.RedDetection(cropFrame, y1)           
            if box is not None:
                text = "Crosswalk"
                mode_status = MODE_CROSSWALK               
            else:
                tic, text, turn, thresh =  lt.LineTracing(threshold, sens, cropFrame, 
                                                          frame, tic, width, height, y1)
                sw.SerialSendCommand(turn, ser, serial_on)                      
        elif mode_status == MODE_CROSSWALK:
            cmd = 0
            sw.SerialSendCommand(cmd, ser, serial_on)
            ret = sw.SerialReceiveResponse(ser, serial_on)           
            if ret == 0:
                tic = time.time()
                wait = 3
                mode_status = MODE_WAIT                
        elif mode_status == MODE_WAIT:
            cropped, box = sign.FindTrafficSign(frame)            
            if time.time() - tic > wait:  
                mode_status = MODE_INFERENCE
        elif mode_status == MODE_INFERENCE:                       
            if cropped is not None:      
                class_id, score, text = sign.ReadTrafficSign(cropped, model, labels)        
                tic = time.time()        
                if score > 0.85:
                    if 'left' in labels[class_id]:
                        mode_status = MODE_MOVE
                        turnMove = turnDegree
                        sw.SerialSendCommand(turnSpeed, ser, serial_on)                         
                    elif 'forward' in labels[class_id]:
                        mode_status = MODE_FORWARD
                        sw.SerialSendCommand(turnSpeed, ser, serial_on)                      
                    elif 'stop' in labels[class_id]:
                        mode_status = MODE_WAIT                                                               
                    elif 'noSign' in labels[class_id]:
                        mode_status = MODE_WAIT                    
                    box = None                 
                else:
                    mode_status = MODE_WAIT                    
            else: 
                mode_status = MODE_WAIT
                wait = .5 
        elif mode_status == MODE_MOVE:               
            if time.time() - tic < alignTime: 
                ret, cmd =  sign.Move(alignTime, tic, turn, True)   
                sw.SerialSendCommand(cmd, ser, serial_on)           
            else:                
                ret, cmd = sign.Move(turnTime, tic, turnMove, False)
                sw.SerialSendCommand(cmd, ser, serial_on)                                
                if ret == 1: 
                    sw.SerialSendCommand(speed, ser, serial_on) 
                    mode_status = MODE_LINE_TRACING
                    tic = time.time()
           
        elif mode_status == MODE_FORWARD:               
                ret, cmd = sign.Move(forwardTime, tic, turn, True)
                sw.SerialSendCommand(cmd, ser, serial_on)                
                if ret == 1: 
                    sw.SerialSendCommand(speed, ser, serial_on)  
                    mode_status = MODE_LINE_TRACING
                    tic = time.time()
                    
        else:
            cmd = '0'      
            sw.SerialSendCommand(cmd, ser, serial_on)      
            
        key = cv2.waitKey(1);        
        if key == ord('q'):
                break      
        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE          
                color = (0,0,255)
                text = "Press 'p' to start"
                box = None               
            elif mode_status == MODE_PAUSE:
                mode_status = MODE_LINE_TRACING
                sw.SerialSendCommand(speed, ser, serial_on)  
                color = (50,255,50)
        
        if box is not None:
            cv2.drawContours(frame,[box],0,color,2) #show the box        
        cv2.putText(frame, text, (30,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
        cv2.imshow('Brain AI Car', frame)

    time.sleep(.5)        
    cmd = 0
    sw.SerialSendCommand(cmd, ser, serial_on)
    
    #end program
    video_capture.release()
    cv2.destroyAllWindows()
    sw.closeSerial(ser, serial_on)
    print('Shutting Down')

### Task 3. 설정 값 변경 및 프로그램 실행

In [7]:
# 설정 값 변경 및 프로그램 실행
video = "http://172.30.1.68:8080/video" # 코드
speed = 500                     # 도로 주행 속도(검정색 라인 따라 주행)
threshold = 100                 # 검정색 인식 임계값(어두울 수록 낮게)
sens = .9                       # 도로 주행 민감도(1 = 민감도 없음, 1.5 = 중간, 2 = 매우 민감)

turnSpeed = 500                 # 회전 주행 속도
turnTime = 3                  # 회전 시간(approx. 3.5 ~ 6)
turnDegree = 60                # 최대 회전 각도(approx. 40 ~ 25)
alignTime = .5                  # 핸들 중앙으로 맞추는 시간

forwardTime = 3                 # 직진 주행 시간

serial_on = True

if __name__ == '__main__': 
    Main()    


 -- Microbit --
Microbit USB detected:  COM4
Loading Model
labels =  ['forward', 'left', 'noSign', 'schoolZone', 'stop']

Press 'q' to quit
Press 'p' to start/pause

Shutting Down
